<h1 style='text-aling:center;color:Navy'>  Big Data Systems - Fall 2021  </h1>
<h1 style='text-aling:center;color:Navy'>  Assignment 6 </h1>

This assignment is focused on Spark Streaming and Kafka.

To complete the assignment, you should complete this notebook by filling in the cells provided.

<b>Submission Deadline: This assignment is due Wednesday, Nov 17 at 11:59 P.M</b>

<div style="font-size:26px;color:#F1F8FC;background-color:red;padding:20px;line-height:30px">Please run the notebook on your local machine but do copy and paste your solution here and submit to Okpy</div>

A few notes before you start:
- Directly sharing answers is not okay, but discussing problems with other students is encouraged.
- You should start early so that you have time to get help if you're stuck.

- Before continuing the assignment, select "Save and Checkpoint" in the File menu and then execute the submit cell below. The result will contain a link that you can use to check that your assignment has been submitted successfully. If you submit more than once before the deadline, we will only grade your final submission. If you mistakenly submit the wrong one, you can head to okpy.org and flag the correct version. There will be another submit cell at the end of the assignment when you finish!

<hr style="border-top: 5px solid orange; margin-top: 1px; margin-bottom: 1px"></hr>
<br>
Before you begin completing the assignment, execute the following cell to load the provided tests.

In [1]:
# Don't change this cell; just run it. 
# When you log-in please hit return (not shift + return) after typing in your email
from client.api.notebook import Notebook
ok = Notebook('Assignment6.ok')

Assignment: Assignment6: Big Stream Data Processing Systems
OK, version v1.15.44



In [2]:
_ = ok.submit()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saving notebook... Saved 'Assignment6.ipynb'.


/opt/conda/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


/opt/conda/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


Submit... 100% complete
Submission successful for user: vijayasimha.bheemireddy@ucdenver.edu
URL: https://kubemaster.ucdenver.pvt:5000/UCDenver/CSCI4951-5951/fa21/assignment6/submissions/e98EYd



<hr style="border-top: 5px solid orange; margin-top: 1px; margin-bottom: 1px"></hr>

<hr style="border-top: 5px solid purple; margin-top: 1px; margin-bottom: 1px"></hr>

<div style="font-size:26px;color:#F1F8FC;background-color:#0095EA;padding:10px;">Exersice 1 - Tweet analysis </div><br>


Before we start with this part of the assignment, we will need to run the python program *tweetread.py* that reads the tweets and streams those to a local port.

On a text editor open the file *tweetread.py*. We need to configure the following parameters. You should obtain the value for each parameter from your tweeter configuration (review the guide).

<pre style="background-color: #ebece4;padding: 10px;border-left: solid 4px orange;">
# Set up your credentials
# add your credentials between ''

consumer_key='1QPpi89KXVAiOul69pqeQucsl'
consumer_secret='tKUlev4ks3W5pOHCBDg02BOggU3yXLvqImXjkzL5eiSIXE0rQa'
access_token ='1447305213321236480-qYqlJzfmswlJS08ffdBPAas7pjqxSQ'
access_secret='enQen0nmzJTpP5ulxsfMmYMWZ9kutevCuhN14hJ2ZG8aH'

</pre>


Also, you might need to change this port in case you receive a *'port xxxx is already in use'* error.
<pre style="background-color: #ebece4;padding: 10px;border-left: solid 4px orange;">
port = 5555
</pre>


**Open a new shell terminal** and execute the following commands.<br>
*We assume you have your downloaded files for this assignment at **~/assignment**, if you are using another path, replace it below.*
<pre style="background-color: #ebece4;padding: 10px;border-left: solid 4px orange;">
cd ~/assignment <br>
python3 tweetread.py
</pre>

Once you have this running, return to the notebook and keep going. 


In [ ]:
import findspark

Update the spark path in the folloiwng cell

In [ ]:
findspark.init('-- Type the path of your spark foler -- ')

In [ ]:
# May cause deprecation warnings, safe to ignore, they aren't errors
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [ ]:
totalTweetCount = 0 #stores the number of tweets
totalCharacters = 0 #stores the total length of all tweets

In [ ]:
# Can only run this once. restart your kernel for any errors.
#to do so click on Kernel, Resrart & Clear Output
sc = SparkContext()

In [ ]:
ssc = StreamingContext(sc, 5 )

- **Notes**: 
    - If you change the port number on tweetread.py, you need to update it here too.<br>
    - Both port numbers (here and tweetread.py) should match. <br>
    - You might need to restart the Kernel before continuing.<br>
    - **Make sure your *tweetread.py* app is running before continue.**

In [ ]:
tweets = ssc.socketTextStream("127.0.0.1", 5555)

In [ ]:
lengths = tweets.map(lambda tweet: len(tweet)) #get the length of tweets

In [ ]:
def updateCounts(rdd):
    try:
        count = rdd.count()
        if(count>0):
            global totalTweetCount
            global totalCharacters
            
            totalTweetCount += count
            totalCharacters += rdd.reduce(lambda len1,len2: len1+len2)
            
        if (totalTweetCount>0):
            print('Total Tweets: ' + str(totalTweetCount) + '\n' +
                  'Total characters: ' + str(totalCharacters) + '\n'
                  'Average Length: ' + str(totalCharacters/totalTweetCount)+
                 '\n------------------------------\n')
        else:
            print('nothing yet')
        
    except Exception as ex:
        print(ex)

In [ ]:
lengths.foreachRDD(lambda rdd: updateCounts(rdd))

**Now start the spark streaming context by running the next cell.**
- You should see an updated output every 5 seconds

*Once you are done, execute the stop command in the cell after.*

In [ ]:
ssc.start()    

In [ ]:
ssc.stop()

<div style="font-size:26px;color:#F1F8FC;background-color:#0095EA;padding:10px;">Exersice 2 - Tweet hashtags analysis </div><br>
In the following cells (add more cells need be), write code that outputs the 10 most popular hashtags in the news category. The tweetread.py filters tweets in the news category for you, so don't worry about that part.

### Use the following configuration for this part of the assignment:
- **Batch size:** 5 seconds
- **Window interval:** 300 seconds (5 minutes)
- **Slide interval:** 10 seconds 


<div style="font-size:26px;color:#F1F8FC;background-color:#0095EA;padding:10px;">Exersice 3 - Kafka </div><br>
Go through the assignment guide for Kafka (see the pdf file in the assignment package). Once you are done reviewing the guide, complete this section.


<div style="font-size:20px;background-color:#0B6713;color:#F1E6E7;padding:10px;">
    To-Do Number 1:
</div>

- Create 2 more consumers by right-clicking on the Consumer class and running it as we did before. 
- Stop the producer and run it again. 
- **Do consumers share the messages? Explain why?**

<div style="font-size:20px;background-color:#0B6713;color:#F1E6E7;padding:10px;">
    To-Do Number 2:
</div>

- Create a new topic (name it topicnew) with replication factor 1 and 3 partitions. 
- Modify the Consumer and Producer classes to read/send messages from/to the *topicnew* rather than the topictest.
- Again, create 3 consumers (run the consumer class 3 times without stopping any of them) and run the producer. 
- **Do consumers share the messages? Explain why?**

<div style="font-size:20px;background-color:#0B6713;color:#F1E6E7;padding:10px;">
    To-Do Number 3:
</div>

- Kill one of the consumers and run the producer again. 
- **What will happen?**
- **Do the two active consumers receive all 10 messages?**